In [ ]:
!pip3 install -q tensorflow_decision_forests

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow_decision_forests as tfdf
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
na_values = train.isna().sum()
na_values = na_values[na_values > 0]
na_values

In [ ]:
train.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
# Lets find out data types of columns
int_col = 0
float_col = 0
object_col = 0
for column in train_sample.columns:
    if train[column].dtypes == 'int64':
        int_col += 1
    elif train[column].dtypes == 'float64':
        float_col += 1
    elif train[column].dtypes == 'object':
        object_col += 1
        
print('# of Integer Columns: ', int_col)
print('# of Float Columns: ', float_col)
print('# of Object Columns: ', object_col)
# Check whether total of in, flaot and object columns is equal to toal columns in dataframe
print(int_col + float_col + object_col == train.shape[1])

In [ ]:
# Lets check whether columns with less than 10 cardinality are present
less_cardinality_cols = []
for column in train.columns.tolist()[:-1]:
    if len(train_sample[column].unique()) <= 10:
        less_cardinality_cols.append(column)
        
print('Number of columns with cardinality less than or euqal to 10: ', len(less_cardinality_cols))

# No column has less cardinality. i.e. no categorical columns present

In [ ]:
# Number of missing values are high, therefore, need to fill the values, instead of dropping them
# Since all the columns having missing values are float, lets fill them with mean
train.fillna(train.mean(), inplace = True)
test.fillna(train.mean(), inplace = True)

In [ ]:
print(train.isnull().values.any())
print(test.isnull().values.any())

In [ ]:
# Select best features having highest kendall correlation
features = []
for i in train.columns.tolist()[:-1]:
    corr = train[[i, 'claim']].corr(method = 'kendall')
    if abs(corr['claim'][i]) > 0.01:
        features.append(i)
print(features)
print(len(features))
features.append('claim')   # In order to be used as label

In [ ]:
train_subset = train[features]
test_subset = test[features[:-1]]

In [ ]:
train_df, val_df = train_test_split(train_subset, test_size = 0.2)

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label = 'claim')
val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val_df, label = 'claim')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_subset)

In [ ]:
model = tfdf.keras.RandomForestModel()
model.compile(metrics = ['accuracy'])
model.fit(train_ds, verbose = True)

In [ ]:
model.evaluate(train_ds)
model.evaluate(val_ds)

In [ ]:
val_pred = model.predict(val_ds)

In [ ]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(val_df['claim'].to_numpy(), val_pred)
print(score)

In [ ]:
test_pred = model.predict(test_ds)

In [ ]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['claim'] = test_pred
submission.head()

In [ ]:
submission.to_csv('submission_2nd September.csv')

In [ ]:
model.summary